In [1]:
import uiautomator2 as u2
import random
import time

In [2]:
d = u2.connect()
d.info

{'currentPackageName': 'com.android.systemui',
 'displayHeight': 2232,
 'displayRotation': 0,
 'displaySizeDpX': 360,
 'displaySizeDpY': 780,
 'displayWidth': 1080,
 'productName': 'YAL-AL10',
 'screenOn': False,
 'sdkInt': 29,
 'naturalOrientation': True}

## 锁屏解锁
自动识别息屏、亮屏状态并解锁, 支持无密码、数字、图案

In [6]:
# 华为系
def unlock(d):
    info = d.info
    if info.get('screenOn') and info.get('currentPackageName') != 'com.android.systemui':
        # 屏幕已点亮且当前包名不是系统UI（意味着已经在主屏幕或者应用内）
        pass
    # elif info.get('screenOn') and info.get('currentPackageName') == 'com.android.systemui':
    #     # 屏幕已点亮且当前包名是系统UI（可能在锁屏界面）
    #     d.press('recent')  # 可能会唤醒密码输入界面
    #     # 模拟解锁手势
    #     d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
    #     time.sleep(0.1)
    #     d.click(0.9, 0.5)
    else:
        # 屏幕未点亮
        d.screen_on()
        d.press('recent')
        # 模拟解锁手势
        d.swipe_points([(250, 1270), (555, 1570), (850, 1890), (865, 1570), (535, 1300), (260, 1565), (265, 1870), (850, 1275)], 0.1)
        time.sleep(0.1)
        d.click(0.9, 0.5)
        


In [7]:
unlock(d)

## 随机打开任意数量个未启动应用
支持三方/系统应用类型限定, 更换请在第8行添加参数pm list pack -3 | -s  
开始打开app前会检测目前已在运行应用并在将运行列表内剔除

In [2]:
def open_random_apps():
    while True:
        try:
            max_apps = int(input('请输入最大将运行的app数量: '))
            break
        except ValueError:
            print('请输入整数!')
    package_list_prefix = str(d.shell('pm list package -3')).split('output=')[1].split(', exit_code')[0].strip("'").split('\\n')[:-1]
    package_list = [pkg.replace('package:', '') for pkg in package_list_prefix if pkg]
    running_app_list = d.app_list_running()
    waiting_app_list = [app for app in package_list if app not in running_app_list]
    for i in range(min(max_apps, len(waiting_app_list))):  # 加入min函数防止溢出
        try:
            waiting_app = random.choice(waiting_app_list)
            d.app_start(waiting_app) # 目前版本的U2该函数有Bug, 所以采用以下方法补救
            pid = d.app_wait(waiting_app, timeout=10)                
            print(f'{waiting_app}进程为{pid}')
        except:
            try:
                d.shell(f"monkey -p {waiting_app} -c android.intent.category.LAUNCHER 1")  # 使用intent过滤器查找到AndroidManifest.xml中的MainACTIVITY, 配合Monkey启动应用,  -p指定包名, -c指定intent类别, 1表示执行时间的数量
                pid = d.app_wait(waiting_app, timeout=10)                
                print(f'{waiting_app}进程为{pid}')
            except Exception as e:
                 print(f'{waiting_app}没在运行!')
                 print(f"捕获到错误：{e}")
        finally:
            waiting_app_list.remove(waiting_app)
            

In [6]:
open_random_apps()

请输入最大将运行的app数量:  10


com.tencent.androidqqmail进程为22511
cmb.pb进程为0
com.zhihu.android进程为23485
com.max.xiaoheihe进程为23966
com.github.uiautomator.test进程为0
ctrip.android.view进程为25361
com.adobe.psmobile进程为25859
cn.edu.zjicm.wordsnet_d进程为26103
hello.litiaotiao.app进程为26528
com.sankuai.meituan进程为26701


## 关闭所有应用

In [6]:
d.app_stop_all()

['com.huawei.security.privacycenter',
 'com.weawow',
 'com.xunmeng.pinduoduo',
 'com.huawei.camera',
 'com.huawei.nearby',
 'com.android.mms',
 'com.huawei.harmonyos.foundation',
 'com.huawei.lbs',
 'com.android.nfc',
 'com.android.systemui',
 'com.huawei.iaware',
 'com.google.android.gms',
 'com.huawei.hidisk',
 'com.huawei.profile',
 'com.android.gallery3d',
 'com.huawei.browser',
 'com.huawei.recsys',
 'com.huawei.intelligent',
 'com.alibaba.android.rimet',
 'com.tencent.mm',
 'com.cz.GJ2X',
 'com.huawei.hiaction',
 'com.huawei.magazine',
 'com.huawei.trustagent',
 'com.huawei.android.thememanager',
 'com.huawei.android.pushagent',
 'com.android.contacts',
 'com.android.phone',
 'com.huawei.himovie',
 'com.baidu.baidutranslate',
 'mark.via.gp',
 'com.huawei.android.launcher',
 'com.huawei.systemserver',
 'com.android.vending',
 'hello.litiaotiao.app',
 'cmb.pb',
 'com.huawei.securityserver',
 'com.xinmang.unzip',
 'com.android.calendar',
 'com.android.bluetooth',
 'com.vnet.tech',
 

## 发送语微信语音消息

In [7]:
def send_wx_audio_msg():
    while True:
        try:
            if d.app_current.get('currentPackageName') != "com.tencent.mm":
                d.app_start('com.tencent.mm')
            
            

BaseError: stat /data/app/com.zhihu.android-lE57VtKtaSmJY68K23g31Q==/base.apk
package:/data/app/com.zhihu.android-lE57VtKtaSmJY68K23g31Q==/split_billboard.apk: no such file or directory

In [43]:
d.app_start("com.bittorrent.client")

BaseError: stat /data/app/com.bittorrent.client-H8kraWadPSbCnaOIgDme5g==/base.apk
package:/data/app/com.bittorrent.client-H8kraWadPSbCnaOIgDme5g==/split_config.arm64_v8a.apk
package:/data/app/com.bittorrent.client-H8kraWadPSbCnaOIgDme5g==/split_config.en.apk
package:/data/app/com.bittorrent.client-H8kraWadPSbCnaOIgDme5g==/split_config.xxhdpi.apk: no such file or directory

In [27]:
d.app_current()

{'package': 'com.bittorrent.client',
 'activity': 'com.bittorrent.app.Main',
 'pid': 2928}